## Chapter 15
# Revisiting character pictures

## Exercises

**15-0.** Compile, execute and test the programs in this chapter.

Implemented in `Picture.h` and `Picture.cpp`.

In [1]:
#include "Picture.h"

In [2]:
#include "../chapter_14/Str.h"

In [3]:
#include "split.h"

In [4]:
#include "../chapter_11/Vec.h"

In [5]:
.L Picture.cpp

In [6]:
.L ../chapter_14/Str.cpp

In [7]:
.L split.cpp

**15-1.** Test your system by writing a program that executes
```
Picture p = // some initial starting picture
Picture q = frame(p)
Picture r = hcat(p, q);
Picture s = vcat(q, r);
cout << frame(hcat(s, vcat(r, q))) << endl;
```

In [8]:
#include <iostream>

In [9]:
using std::cout;

In [10]:
using std::endl;

In [11]:
Vec<Str> strings = split("and now for something completely different");

Picture p = strings;
Picture q = frame(p);
Picture r = hcat(p, q);
Picture s = vcat(q, r);
Picture final = frame(hcat(s, vcat(r, q)));
cout << final << endl;
// Note that this has been rerun _after_ implementing all exercises.
// The only functional different here is the centered `hcat` operation.

****************************************************
*                                                  *
* **************                    ************** *
* *            *                    *            * *
* * and        *          and       * and        * *
* * now        *          now       * now        * *
* * for        *          for       * for        * *
* * something  *          something * something  * *
* * completely *          completely* completely * *
* * different  *          different * different  * *
* *            *                    *            * *
* **************                    ************** *
*           ****************************           *
*           *            **            *           *
* and       * and        ** and        *           *
* now       * now        ** now        *           *
* for       * for        ** for        *           *
* something * something  ** something  *           *
* completely* completely ** completely *      

(std::__1::basic_ostream &) @0x7fffa7cc4660


**15-2.** Reimplement the `Frame_Pic` class so that the frame uses three different characters: one for the corners, another for the top and bottom borders, and a third for the side borders.

I implemented this with default arguments to the `frame` method, which forwards them to the `Frame_Pic` constructor (with no defaults):

`Picture frame(const Picture&, const char='*', const char='*', const char='*');
`

Example:

In [12]:
Picture special_edges = frame(p, '^', '-', '|');
cout << special_edges << endl;

^------------^
|            |
| and        |
| now        |
| for        |
| something  |
| completely |
| different  |
|            |
^------------^



(std::__1::basic_ostream &) @0x7fffa7cc4660


**15-3.** Give users the option to specify what characters to use for these border characters.

(Done above!)

**15-4.** Add an operation to `reframe` a `Picture`, which changes the frame characters. The operation should change all of the frames in the interior picture.

I implemented this by adding a new `virtual` function to `Pic_base` and implementing it in each sublass recursively on its `Ptr<Pic_base>` members:

```
virtual void Pic_base::reframe(const char corner_char, const char top_bottom_char, const char sides_char) { } // only Frame_Pic actually implements.
...
const Picture& reframe(const Picture&, const char='*', const char='*', const char='*');
```

The most interesting case is the `Frame_pic` implementation, which changes its internal members and those of its child (which may be non-`Frame` `Pic_base`s that contain other `Frame_pic`s):

```
    void reframe(const char corner_char, const char top_bottom_char, const char sides_char) {
        this->corner_char = corner_char;
        this->top_bottom_char = top_bottom_char;
        this->sides_char = sides_char;
        p->reframe(corner_char, top_bottom_char, sides_char);
    }
```

Examples with different numbers of overloaded arguments:

In [13]:
cout << reframe(final, '^') << endl;
cout << reframe(final, '^', '-') << endl;
cout << reframe(final, '^', '-', '|') << endl;

^**************************************************^
*                                                  *
* ^************^                    ^************^ *
* *            *                    *            * *
* * and        *          and       * and        * *
* * now        *          now       * now        * *
* * for        *          for       * for        * *
* * something  *          something * something  * *
* * completely *          completely* completely * *
* * different  *          different * different  * *
* *            *                    *            * *
* ^************^                    ^************^ *
*           ^************^^************^           *
*           *            **            *           *
* and       * and        ** and        *           *
* now       * now        ** now        *           *
* for       * for        ** for        *           *
* something * something  ** something  *           *
* completely* completely ** completely *      

(std::__1::basic_ostream &) @0x7fffa7cc4660


**15-5.** Reimplement `HCat_Pic` so that when pictures of a different size are concatenated, the shorter one is centered in the space consumed by the longer one. That is, if we horizontally concatenate two pictures, one of which is four lines long and the other is two lines long, the first and last rows of the output picture will be blank on the side of the shorter picture. What can we now conclude about the necessity of the tests between `row` and `0`?

Implementation (could be DRY'd up, be eh...):

```
void HCat_Pic::display(ostream& os, ht_sz row, bool do_pad) const {
    if (left->height() < right->height()) {
        ht_sz half_height_diff = (right->height() - left->height()) / 2;
        if (row < half_height_diff || row >= right->height() - half_height_diff) {
            pad(os, 0, left->width());            
        } else {
            left->display(os, row - half_height_diff, do_pad || row - half_height_diff < right->height());
        }
    } else {
        left->display(os, row, do_pad || row < right->height());
    }

    if (left->height() > right->height()) {
        ht_sz half_height_diff = (left->height() - right->height()) / 2;
        if (row < half_height_diff || row >= left->height() - half_height_diff) {
            pad(os, 0, right->width());            
        } else {
            right->display(os, row - half_height_diff, do_pad);
        }
    } else {
        right->display(os, row, do_pad);
    }
}
```
I can't really decipher the question about the `row` and `0` comparison. The only such comparison I see like that at all in the entire implementation is within `Frame_pic::display`, which is irrelevant when considering the general case.  I think what they're getting at is that one could choose to implement this in a different way that delegates the row-padding to another function that accepts rows that could be `<= 0`...?

Example of this function in action:

In [14]:
cout << hcat(p, frame(p)) << endl;

          **************
          *            *
and       * and        *
now       * now        *
for       * for        *
something * something  *
completely* completely *
different * different  *
          *            *
          **************



(std::__1::basic_ostream &) @0x7fffa7cc4660


In [15]:
cout << p << endl;

and
now
for
something
completely
different



(std::__1::basic_ostream &) @0x7fffa7cc4660


**15-6.** The `Vec` and `Str` classes that we developed in Chapters 11 and 12 are powerful enough to be used to implement `Picture`s. Reimplement the material in this chapter to use `Vec<Str>` instead of `vector<string>`, and test your implementation.

Done! All examples have been run using this implementation instead of making a copy of everything.